### Import Spring 2022 Data

Starting off, we are going to use pandas to read the downloaded `Spring 2022 Outcome Report` to a dataframe. This cell will display that dataframe and give us an opportunity to take a first look at the data.

In [1]:
!pip install pandas

In [2]:
import pandas as pd
import os

file_path = os.getcwd() + '\\1_Landing\\Spring 2022 Outcome Report.csv'

data = pd.read_csv(file_path)
dataframe = pd.DataFrame(data)

dataframe


,student name,student id,student sis id,assessment title,assessment id,assessment type,submission date,submission score,learning outcome name,learning outcome id,...,assignment url,learning outcome friendly name,learning outcome points possible,learning outcome mastery score,learning outcome mastered,learning outcome rating,learning outcome rating points,account id,account name,enrollment state
0,NaN,13328,NaN,My Theology Paper: Comparing and Contrasting ...,422281,assignment,2022-04-18T13:29:48-06:00,200.0,GE_CF2_U1415 Christian Formation 2,400,...,https://nnu.instructure.com/courses/28474/assi...,Christian Formation 2,3.0,2.0,1.0,Exemplary: Articulate distinctive characterist...,3.0,107,Undergraduate Philosophy and Religion,active
1,NaN,17704,NaN,My Theology Paper: Comparing and Contrasting ...,422281,assignment,2022-05-01T21:26:54-06:00,160.0,GE_CF2_U1415 Christian Formation 2,400,...,https://nnu.instructure.com/courses/28474/assi...,Christian Formation 2,3.0,2.0,0.0,Minimum: List the characteristics of Wesleyan ...,1.0,107,Undergraduate Philosophy and Religion,active
2,NaN,20155,NaN,My Theology Paper: Comparing and Contrasting ...,422281,assignment,2022-05-02T22:17:02-06:00,190.0,GE_CF2_U1415 Christian Formation 2,400,...,https://nnu.instructure.com/courses/28474/assi...,Christian Formation 2,3.0,2.0,1.0,Exemplary: Articulate distinctive characterist...,3.0,107,Undergraduate Philosophy and Religion,active
3,NaN,21601,NaN,My Theology Paper: Comparing and Contrasting ...,422281,assignment,2022-05-02T22:36:29-06:00,180.0,GE_CF2_U1415 Christian Formation 2,400,...,https://nnu.instructure.com/courses/28474/assi...,Christian Formation 2,3.0,2.0,1.0,Target: Compare and contrast Wesleyan theology...,2.0,107,Undergraduate Philosophy and Religion,active
4,NaN,24034,NaN,My Theology Paper: Comparing and Contrasting ...,422281,assignment,2022-05-02T20:32:54-06:00,190.0,GE_CF2_U1415 Christian Formation 2,400,...,https://nnu.instructure.com/courses/28474/assi...,Christian Formation 2,3.0,2.0,1.0,Exemplary: Articulate distinctive characterist...,3.0,107,Undergraduate Philosophy and Religion,active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1494,NaN,40928,NaN,Argument Essay Final Draft,466450,assignment,2022-05-02T00:12:44-06:00,198.0,GE_IP4_U2021,3647,...,https://nnu.instructure.com/courses/28700/assi...,Intellectual and Practical Skills 4,NaN,NaN,1.0,Exemplary: Research multiple sources of infor...,NaN,87,Undergraduate Langauge & Lit,active
1495,NaN,41052,NaN,Argument Essay Final Draft,464908,assignment,2022-05-03T23:26:37-06:00,140.0,GE_IP4_U2021,3647,...,https://nnu.instructure.com/courses/29652/assi...,Intellectual and Practical Skills 4,NaN,NaN,0.0,Outcome Not Met,NaN,87,Undergraduate Langauge & Lit,active
1496,NaN,41073,NaN,Argument Essay Final Draft,466450,assignment,2022-04-29T23:58:24-06:00,160.0,GE_IP4_U2021,3647,...,https://nnu.instructure.com/courses/28700/assi...,Intellectual and Practical Skills 4,NaN,NaN,1.0,Target: Research information in response to c...,NaN,87,Undergraduate Langauge & Lit,active
1497,NaN,41102,NaN,Argument Essay Final Draft,466450,assignment,2022-04-29T21:40:59-06:00,175.0,GE_IP4_U2021,3647,...,https://nnu.instructure.com/courses/28700/assi...,Intellectual and Practical Skills 4,NaN,NaN,1.0,Target: Research information in response to c...,NaN,87,Undergraduate Langauge & Lit,active


### Data Manipulation
I wrote a lengthy description of the data types we kept exactly as it, which ones we engineered, and which ones we got rid of, but it got deleted.

In [4]:
## Specify what indexes we care about
indexes_of_interest = [1,6,8,14,16,17,25,26]


student_ids = []            #index=1
submission_dates = []       #index=6
learning_outcome_codes = []  #index=7
learning_outcome_years = []
course_names =[]            #index=14
semesters = []
course_sis_ids = []         #index=16
section_names = []          #index=17
learning_outcome_rating = []#index=25
learning_outcome_points = []#index=26
adjusted_learning_outcomes = []
final_outcome_info = []

college = []

compliance = []

## Read data from the dataframe
## For some columns, the information will be manipulated
for index, rows in dataframe.iterrows():
    student_ids.append(rows[1])

    ## Manipulate the date, so that we only get YYYY-MM-DD
    submission_dates.append(str(rows[6]).split('T')[0])

    ## Manipulate the learning outcome name, seperating into code and year
    learning_outcome_name = rows[8].split('_')
    if(len(learning_outcome_name) > 2):
        learning_outcome_years.append(learning_outcome_name[2].split(' ')[0])
        learning_outcome_codes.append(learning_outcome_name[1])
    elif(len(learning_outcome_name) > 1):
        learning_outcome_codes.append(learning_outcome_name[1])
        learning_outcome_years.append('NaN')
    else:
        learning_outcome_codes.append(learning_outcome_name[0])
        learning_outcome_years.append('NaN')


    ## Original method, but just duplicates the same information 3 times
    #course_names.append(rows[14])
    course_sis_ids.append(rows[16])
    #section_names.append(rows[17])


    ## Slice the course code into 3 pieces of information: semester, course code, section code
    course_id = rows[17].split(' ')[-1].split('_')
    semesters.append(course_id[0])
    course_names.append(course_id[1])
    section_names.append(course_id[2])
    

    # Manipulate the rating, getting rid of everything except for the actual value
    outcome_rating = str(rows[25]).split(':')[0]
    learning_outcome_rating.append(outcome_rating)
    
    
    # Leave points as is, maybe convert to int
    outcome_points = rows[26]
    learning_outcome_points.append(outcome_points)




    # Calculate an adjusted learning outcome score
    if outcome_points > 0 and outcome_points <= 3.0:
        adjusted_learning_outcomes.append(outcome_points)
    elif outcome_points > 3.0 and outcome_points <= 5.0:
        adjusted_learning_outcomes.append(-1)
    else:
        if(outcome_rating == 'Exemplary'):
            adjusted_learning_outcomes.append(3.0)
        elif (outcome_rating == 'Target'):
            adjusted_learning_outcomes.append(2.0)
        elif(outcome_rating == 'Minimum'):
            adjusted_learning_outcomes.append(1.0)
        elif (outcome_rating == 'Outcome Not Met' or outcome_rating == 'Outcome not met.'):
            adjusted_learning_outcomes.append(0.0)
        else:
            adjusted_learning_outcomes.append(-2)


    # Based on the outcome score, provide an actual rating
    if(adjusted_learning_outcomes[index] == 3.0):
        final_outcome_info.append('Exemplary')
    elif(adjusted_learning_outcomes[index] == 2.0):
        final_outcome_info.append('Target')
    elif(adjusted_learning_outcomes[index] == 1.0):
        final_outcome_info.append('Minimum')
    elif(adjusted_learning_outcomes[index] == 0.0):
            final_outcome_info.append('Fail')
    elif(adjusted_learning_outcomes[index] == -2):
        final_outcome_info.append('No Data Collected')
    elif(adjusted_learning_outcomes[index] == -1):
        final_outcome_info.append('Invalid Entry - Distinguished/Commendable')
    else:
        final_outcome_info.append('Unknown Error')


    ## Add in college code
    theology = ['THEO', 'BIBL', 'PHIL', 'PRTH']
    nat_sci = ['COMP', 'BIOL', 'CHEM', 'ENGR', 'PHYS', 'KINE', 'MILS']
    art_hum = ['CLTA', 'COMM' , 'HUMN', 'ARDE', 'MUSC', 'ENGL', 'GEOG', 'POLS', 'HIST' , 'FREN', 'SPAN']
    business = ['ECON', 'BSNS']
    behav_social = ['PSYC', 'SOWK', 'CRIM']
    education = ['EDUC']
    nursing = ['NURS']

    course_code = course_names[index][0:4]
    if(course_code in theology):
        college.append('College of Theology & Christian Ministry')
    elif(course_code in nat_sci):
        college.append('College of Natural and Applied Sciences')
    elif(course_code in art_hum or course_code == 'HNRS' or course_code == 'INTD'):
        college.append('College of Arts and Humanites')
    elif(course_code in business):
        college.append('College of Business')
    elif(course_code in behav_social):
        college.append('College of Behavior and Social Sciences')
    elif(course_code in education):
        college.append('College of Educaton')
    elif(course_code in nursing):
        college.append('College of Nursing')
    else:
        college.append('We dont know')

    ## Based on outcome ratings, lets add outcome score
    if (adjusted_learning_outcomes[index] == 3 or adjusted_learning_outcomes[index] == 2 
    or adjusted_learning_outcomes[index] == 1 or adjusted_learning_outcomes[index] == 0):
        compliance.append('Outcome assessed')
    elif (adjusted_learning_outcomes[index] == -2):
        compliance.append('Outcome attached, no assessment')
    elif (adjusted_learning_outcomes[index] == -1):
        compliance.append('Outcome attached, invalid assessment')
    else:
        compliance.append('OUtcome not attached')


### Create and Export New Dataframe
Since we have now cleaned and staged the data for visualization, we want to create a new dataframe, then save it as a .csv file

In [6]:
new_data = pd.DataFrame(list(zip(submission_dates, learning_outcome_codes, learning_outcome_years, course_sis_ids, semesters, course_names, section_names, learning_outcome_rating, learning_outcome_points, adjusted_learning_outcomes, final_outcome_info, college, compliance))
,index = student_ids, columns=('submission_date', 'outcome code', 'outcome year', 'course_id', 'semester', 'course', 'section', 'outcome rating', 'outcome score', 'corrected outcome score', 'corrected outcome rating', 'college' , 'compliance'))

new_data.to_csv('2_Staging\\Spring 2022 Outcome Report 2.0.csv')

new_data

,submission_date,outcome code,outcome year,course_id,semester,course,section,outcome rating,outcome score,corrected outcome score,corrected outcome rating,college,compliance
13328,2022-04-18,CF2,U1415,SP2022_THEO2100_03,SP2022,THEO2100,03,Exemplary,3.0,3.0,Exemplary,College of Theology & Christian Ministry,Outcome assessed
17704,2022-05-01,CF2,U1415,SP2022_THEO2100_03,SP2022,THEO2100,03,Minimum,1.0,1.0,Minimum,College of Theology & Christian Ministry,Outcome assessed
20155,2022-05-02,CF2,U1415,SP2022_THEO2100_03,SP2022,THEO2100,03,Exemplary,3.0,3.0,Exemplary,College of Theology & Christian Ministry,Outcome assessed
21601,2022-05-02,CF2,U1415,SP2022_THEO2100_03,SP2022,THEO2100,03,Target,2.0,2.0,Target,College of Theology & Christian Ministry,Outcome assessed
24034,2022-05-02,CF2,U1415,SP2022_THEO2100_03,SP2022,THEO2100,03,Exemplary,3.0,3.0,Exemplary,College of Theology & Christian Ministry,Outcome assessed
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40928,2022-05-02,IP4,U2021,SP2022_ENGL1030B_02,SP2022,ENGL1030B,02,Exemplary,NaN,3.0,Exemplary,College of Arts and Humanites,Outcome assessed
41052,2022-05-03,IP4,U2021,SP2022_ENGL1030B_01,SP2022,ENGL1030B,01,Outcome Not Met,NaN,0.0,Fail,College of Arts and Humanites,Outcome assessed
41073,2022-04-29,IP4,U2021,SP2022_ENGL1030B_02,SP2022,ENGL1030B,02,Target,NaN,2.0,Target,College of Arts and Humanites,Outcome assessed
41102,2022-04-29,IP4,U2021,SP2022_ENGL1030B_02,SP2022,ENGL1030B,02,Target,NaN,2.0,Target,College of Arts and Humanites,Outcome assessed


### Create and Export an even better Dataframe
Cole's version, corrected and cleaned to make data visualization better.

**Full description coming soon**

In [7]:
coles_format = pd.DataFrame(list(zip(submission_dates, learning_outcome_codes, learning_outcome_years, semesters, course_names, section_names, adjusted_learning_outcomes, final_outcome_info, college, compliance))
,index = student_ids, columns=('submission_date', 'outcome code', 'outcome year', 'semester', 'course', 'section', 'outcome score', 'outcome rating', 'college', 'compliance'))

coles_format.to_csv('2_Staging\\Spring 2022 Outcome Report 2.1.csv')

coles_format

,submission_date,outcome code,outcome year,semester,course,section,outcome score,outcome rating,college,compliance
13328,2022-04-18,CF2,U1415,SP2022,THEO2100,03,3.0,Exemplary,College of Theology & Christian Ministry,Outcome assessed
17704,2022-05-01,CF2,U1415,SP2022,THEO2100,03,1.0,Minimum,College of Theology & Christian Ministry,Outcome assessed
20155,2022-05-02,CF2,U1415,SP2022,THEO2100,03,3.0,Exemplary,College of Theology & Christian Ministry,Outcome assessed
21601,2022-05-02,CF2,U1415,SP2022,THEO2100,03,2.0,Target,College of Theology & Christian Ministry,Outcome assessed
24034,2022-05-02,CF2,U1415,SP2022,THEO2100,03,3.0,Exemplary,College of Theology & Christian Ministry,Outcome assessed
...,...,...,...,...,...,...,...,...,...,...
40928,2022-05-02,IP4,U2021,SP2022,ENGL1030B,02,3.0,Exemplary,College of Arts and Humanites,Outcome assessed
41052,2022-05-03,IP4,U2021,SP2022,ENGL1030B,01,0.0,Fail,College of Arts and Humanites,Outcome assessed
41073,2022-04-29,IP4,U2021,SP2022,ENGL1030B,02,2.0,Target,College of Arts and Humanites,Outcome assessed
41102,2022-04-29,IP4,U2021,SP2022,ENGL1030B,02,2.0,Target,College of Arts and Humanites,Outcome assessed


## Check how many courses that we actually have data for

In [8]:
alignment_path = os.getcwd() + '\\1_Landing\\GE Outcome_Course Alignment Worksheet.xlsx'

ge_alignment = pd.read_excel(alignment_path)

ge_alignment


,General Education Outcome/Course Alignment by Outcomes TUG,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,Directions: Find the outcome and then search i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CF1,CF2,CF3,CF4,SC1,SC2,SC3,SC4,HU1,HU2,HU3,HU4,IP1,IP2,IP3,IP4
2,BIBL1100,THEO2100,BIBL3220,HNRS2550,BIOL1010,BIOL1010L,CLTA1550,KINE1000,ARDE2020,HUMN1020,BIBL3810,HIST1030,COMM1210,ENGL3015,BSNS2330,ENGL1030A
3,NaN,NaN,BIBL3230,PHIL2010,BIOL1030,BIOL1030L,CLTA3180,KINE1010,ARDE3250,HNRS2550,BIOL1030,HIST1040,COMM3211,ENGL2610W,COMP1220,ENGL1030B
4,NaN,NaN,BIBL3260,PHIL2020,BIOL1040,BIOL1040L,COMM2030,KINE1020,ARDE3340,ARDE4950,BIOL3520L,HIST1070,NaN,ENGL3020W,KINE4520,HNRS2560
5,NaN,NaN,BIBL3270,PHIL2030,BIOL1060,BIOL1060L,COMM3200,KINE1030,ARDE3750,BIOL4970,BSNS4600,HIST2030,NaN,ENGL3040W,MATH1100,NaN
6,NaN,NaN,BIBL3810,PHIL3010,BIOL2010,BIOL2010L,ECON1050,KINE2640,ARDE3950,BIOL4980,CLTA1550,HIST2040,NaN,ENGL3210W,MATH1300,NaN
7,NaN,NaN,HNRS3560,PHIL3080,BIOL2030,BIOL2030L,ECON2010,KINE2770,MUSC1010,BSNS4980,CLTA3180,HIST3390,NaN,ENGL3250W,MATH1400,NaN
8,NaN,NaN,THEO3410,PHIL3520,BIOL2220,BIOL2220L,ECON2410,MILS2500,MUSC2020,CHEM4980,COMM1050,HIST3480,NaN,ENGL3310W,MATH2010,NaN
9,NaN,NaN,THEO3540,PHIL3600,BIOL2230,BIOL2230L,ECON2420,NaN,MUSC3260,COMM4980,COMM3200,HNRS2560,NaN,ENGL3320W,MATH2240,NaN


In [9]:
list_of_courses = []

for (index_label, row_series) in ge_alignment.iterrows():
    for value in row_series.values:
        if(len(str(value)) > 4 and len(str(value)) < 12):
            if str(value) not in list_of_courses:
                list_of_courses.append(str(value))


In [10]:
assessed_courses = []

for (index_label, row_series) in coles_format.iterrows():
    if(row_series[4] in list_of_courses and row_series[4] not in assessed_courses):
        assessed_courses.append(str(row_series[4]))

print(list_of_courses)
print(assessed_courses)

print(str(len(assessed_courses)) + ' out of ' + str(len(list_of_courses)) + ' courses have been assessed')



    

['BIBL1100', 'THEO2100', 'BIBL3220', 'HNRS2550', 'BIOL1010', 'BIOL1010L', 'CLTA1550', 'KINE1000', 'ARDE2020', 'HUMN1020', 'BIBL3810', 'HIST1030', 'COMM1210', 'ENGL3015', 'BSNS2330', 'ENGL1030A', 'BIBL3230', 'PHIL2010', 'BIOL1030 ', 'BIOL1030L', 'CLTA3180', 'KINE1010', 'ARDE3250', 'BIOL1030', 'HIST1040', 'COMM3211', 'ENGL2610W', 'COMP1220', 'ENGL1030B', 'BIBL3260', 'PHIL2020', 'BIOL1040', 'BIOL1040L', 'COMM2030', 'KINE1020', 'ARDE3340', 'ARDE4950', 'BIOL3520L', 'HIST1070', 'ENGL3020W', 'KINE4520', 'HNRS2560', 'BIBL3270', 'PHIL2030 ', 'BIOL1060', 'BIOL1060L', 'COMM3200', 'KINE1030', 'ARDE3750', 'BIOL4970', 'BSNS4600', 'HIST2030', 'ENGL3040W', 'MATH1100', 'PHIL3010', 'BIOL2010', 'BIOL2010L', 'ECON1050', 'KINE2640', 'ARDE3950', 'BIOL4980', 'HIST2040', 'ENGL3210W', 'MATH1300', 'HNRS3560', 'PHIL3080', 'BIOL2030', 'BIOL2030L', 'ECON2010', 'KINE2770', 'MUSC1010', 'BSNS4980', 'HIST3390', 'ENGL3250W', 'MATH1400', 'THEO3410', 'PHIL3520', 'BIOL2220', 'BIOL2220L', 'ECON2410', 'MILS2500', 'MUSC2020'